# Planning

1. Make a general template for geant4, which should be just including the most basic elements, so it can carry out more flexible setting

2. Setting up following setting in different repo
- Na$^22$ source
- A single CsI crystal with SiPM attached to it. This setting is for simulating scintillator detector more realisticly
- A Xenon gas with shielding plate, this setting is for simulating more realistic positornium generator
- Quantum dot dark box setting (optional)

3. For physics list
- Verify from tom's positronium generation mechanism (how to import custom physics list)

4. For the generated data file, also sensitive detector setting
- It should include: 
  - Scintillator detector, which count the number of optical photon corresponding to the event.
  - Ideal detector, measuring the initial and final energy that a particular particle deposit inside the detector.

5. For the particle generation, which should include
- Na$^{22}$ source
- Cosmic ray
- Other customized stuff

# Geant4 study plan
- Defining Hit class, Hit collection
Potential resource
  - https://agenda.infn.it/event/7698/contributions/69590/attachments/50573/59738/Pandola_-_Kernel.pdf
  - https://indico.cern.ch/event/294651/sessions/55918/attachments/552022/760640/UserActions.pdf
  - https://indico.cern.ch/event/782305/contributions/3256028/attachments/1793321/2922650/2b_UserActions.pdf
  - https://www.youtube.com/watch?v=SVoddk8S-mk
  - Novice Example N02 (Learn HIT collection to store datas)
  - Novice Example N04 (For fun, Pythia primary event generator)
  - https://jeffersonlab.github.io/geant4-tutorials/HandsOn3/index.html
  - https://geant4-userdoc.web.cern.ch/UsersGuides/ForApplicationDeveloper/html/Detector/hit.html#hit
- Defining Digit, Digitization
Potential resource
  - Gamma Ray Space Telescope (advanced example)
  - https://indico.cern.ch/event/75452/contributions/2089769/attachments/1049576/1496253/HitsAndDigits.pdf
- Understanding multithreading
Potential resource
  - https://indico.fnal.gov/event/57914/contributions/260765/attachments/164432/218103/g4-mt-tasking-syjun.pdf

- Understanding Scintillator detector
Potential resource
  - Novice Example N06 (Scintillation materials and scintillation spectra specification)
  - extended/OpNovice
  - Underground physics (advanced example)
  - CaTS (advanced example, using G4Opticks to do optical physics with GPU)
  - https://indico.cern.ch/event/1052654/contributions/4525304/attachments/2310901/3932510/G4RandD_v4.pdf
- Extended new physics
Potential resource
  - Novice Example N04 (For fun, Pythia primary event generator)
  - exoticphysics (extended example)